In [1]:
import os,sys
import random
from sample import Sample
from LoadData import get_train_data
from LoadData import get_eval_data
from MyDataSet import MyDataSet
from MyDataSet import collate_func
from word_vector_train import SigmoidBinCELoss
from word_vector_train import train_word_vector
from CSM import CSM
import collections
import torch 
import torch.nn as nn 
import torch.utils.data
import sys
import os
import random
import math
import torch.nn.functional as F
import pickle
random.seed(1)   #设置随机种子

BATCH_SIZE=128
root_path = "./"

## 重要：首先你要生成好一个word_vocab。然后执行下面的代码读取这个word_vocab。之后每次都用统一的词表，不然会序号混乱。word_vocab就是一个dict。它的key是词，value是index

In [2]:
with open('word_vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)
wd2Idx = vocab.copy()
idx2Wd = {vocab[word] :word for word in vocab.keys() }

## 重要：既然你们要加入分隔符，那么也要对分隔符进行采样。所以你要修改LoadData.py文件中的get_train_data函数。把分隔符加进去。

In [3]:
#读取训练集
poem_line_lst5, poem_line_lst7, poem_vec_lst5, poem_vec_lst7 = get_train_data( 
    "qtrain", wd2Idx)


这里是对词表等进行简单的测试

In [6]:
print(vocab['<S>'])

1511


In [7]:
print(idx2Wd[1511])

<S>


In [9]:
print(wd2Idx['<E>'])
print(poem_line_lst5[0])

6074
['<S>', '持', '鹬', '蚌', '谋', '壮', '贪', '蝉', '鹊', '意', '深', '渔', '人', '一', '拱', '手', '弹', '者', '笑', '依', '林', '<E>']


In [11]:
#进行采样
poem_sample = Sample(vocab, idx2Wd, wd2Idx, poem_vec_lst5, poem_vec_lst7, max_window_size=3, noise_num=2)
centers, all_contexts, noises = poem_sample.get_sample_data()

In [13]:
print(centers[0])
print(all_contexts[0])
print(idx2Wd[1511]), print(idx2Wd[2910])

1511
[2910]
<S>
持


(None, None)

In [14]:
#把中心词，背景词，噪声词加入预训练词向量的dataset和dataloader中
word_vec_data_set = MyDataSet(centers, all_contexts, noises)#把得到的词放到MyDataSet中
word_vec_data_iter = torch.utils.data.DataLoader(word_vec_data_set, BATCH_SIZE, shuffle=True, collate_fn=collate_func, num_workers=0)

In [15]:
#开始进行预训练词向量

embed_size = 200
learn_rate = 0.001
epochs = 10
vocab_size = len(vocab)
loss = SigmoidBinCELoss()
#构建词词嵌入层，训练其中的词向量
word_vec_net = nn.Sequential(nn.Embedding(vocab_size, embed_size),
                     nn.Embedding(vocab_size, embed_size))
#训练
train_word_vector(word_vec_net, word_vec_data_iter, learn_rate, loss, epochs)
#训练完成后，中心词向量word_vec_net[0]可作为词的表征向量

train on cuda
save model,in epoch 0
epoch 0, loss 2.50, time 181.35s
save model,in epoch 1
epoch 1, loss 0.85, time 213.57s
save model,in epoch 2
epoch 2, loss 0.65, time 207.84s
save model,in epoch 3
epoch 3, loss 0.59, time 187.51s
save model,in epoch 4
epoch 4, loss 0.56, time 181.75s
save model,in epoch 5
epoch 5, loss 0.55, time 181.72s
save model,in epoch 6
epoch 6, loss 0.54, time 181.99s
save model,in epoch 7
epoch 7, loss 0.54, time 183.00s
save model,in epoch 8
epoch 8, loss 0.53, time 181.81s
save model,in epoch 9
epoch 9, loss 0.53, time 181.69s


以下是使用预训练的词向量的例子

In [16]:
 #首先，读取已保存好的模型参数
word_vec_net.load_state_dict(torch.load('net_epoch_9.pth'))

#这是使用预训练词向量的demo
#使用预训练的词向量。例如：
csm = CSM(vocab_size, embed_size, scentence_size=5)
csm.embedding.weight.data.copy_(word_vec_net[0].weight.data)#word_vec_net[0]中的数据，就是预训练的词向量


tensor([[ 0.1055, -0.1768,  0.9093,  ..., -0.5408,  0.1471,  0.0205],
        [ 0.6076,  0.9130, -0.2320,  ..., -1.0624, -0.2578, -1.1538],
        [-2.3329,  0.1328,  0.3395,  ...,  0.3982,  1.4768,  0.0994],
        ...,
        [-0.1787, -0.3751, -0.1778,  ...,  2.7739, -0.5851, -0.5034],
        [-1.1934, -1.7688, -1.6601,  ...,  0.3613, -1.2547, -2.8550],
        [-1.1588,  1.3626,  0.4672,  ...,  0.1904,  0.0054, -0.8986]])

In [2]:
get_similar_token('好', 5, word_vec_net[0].weight.data, wd2Idx, idx2Wd)  

NameError: name 'word_vec_net' is not defined